In [1]:
import glob

train_dir = '../input/crop_train'
test_dir = '../input/test'
test_files = sorted(glob.glob(test_dir+'/*'))
train_files = sorted(glob.glob(train_dir+'/*/*'))
from keras.models import load_model
model = load_model('best_xception_crop.h5')

Using TensorFlow backend.


In [2]:
import os
import numpy as np
from PIL import Image
import skimage
import skimage.io
import scipy.misc
import imageio
import warnings; warnings.simplefilter('ignore')
CROP_LEN = 112


def center_crop(im_array):
    center_x = im_array.shape[0] // 2
    center_y = im_array.shape[1] // 2
    half_crop = CROP_LEN // 2
    a = center_x - half_crop
    b = a + CROP_LEN
    c = center_y - half_crop
    d = c + CROP_LEN
    new_im_array = im_array[a:b,c:d,:]
    return new_im_array


def get_tta_img(img_path):
    im_array = np.array(Image.open((img_path)), dtype="uint8")
    img_x = im_array.shape[0]
    img_y = im_array.shape[1]
    im_array = im_array/255.0
    
    img_list = []
    img_list.append(center_crop(im_array))
    for i in range(5):
        for j in range(5):
            a = i*100
            b = a + 112
            c = j*100
            d = c + 112
            img_list.append(im_array[a:b,c:d,:])
    return np.array(img_list)
print('def done')

def done


In [3]:
list_classes = [
 'Sony-NEX-7',
 'Motorola-X',
 'HTC-1-M7',
 'Samsung-Galaxy-Note3',
 'Motorola-Droid-Maxx',
 'iPhone-4s',
 'iPhone-6',
 'LG-Nexus-5x',
 'Samsung-Galaxy-S4',
 'Motorola-Nexus-6']

dict_cls = {}
for i,k in enumerate(list_classes):
    dict_cls[k] = i

tmp_cnt = 0
x_data,y_data = [],[]
for img_p in train_files:
    tmp_cnt += 1
    if tmp_cnt % 1000 == 0:
        print('processing',tmp_cnt)
    
    tta_imgs = get_tta_img(img_p)
    preds = list(np.mean(model.predict(tta_imgs),axis=0))
    max_idx = np.argmax(preds)
    
    # add x,y
    res = preds + [max_idx]
    x_data.append(res)
    tmp_y = img_p.split('/')[-2]
    y_data.append(dict_cls[tmp_y])

print('train_x done')
print(x_data[0])
print(y_data[0])

tmp_cnt = 0
test_x_data = []
for img_p in test_files:
    tmp_cnt += 1
    if tmp_cnt % 1000 == 0:
        print('processing',tmp_cnt)
    
    tta_imgs = get_tta_img(img_p)
    preds = list(np.mean(model.predict(tta_imgs),axis=0))
    max_idx = np.argmax(preds)
    
    # add x,y
    res = preds + [max_idx]
    test_x_data.append(res)

print('test x done')
print(test_x_data[0])
    




processing 1000
processing 2000
processing 3000
processing 4000
processing 5000
processing 6000
processing 7000
processing 8000
processing 9000
processing 10000
processing 11000
processing 12000
processing 13000
processing 14000
processing 15000
processing 16000
processing 17000
processing 18000
processing 19000
processing 20000
processing 21000
processing 22000
processing 23000
processing 24000
processing 25000
processing 26000
processing 27000
processing 28000
processing 29000
processing 30000
processing 31000
processing 32000
processing 33000
processing 34000
processing 35000
processing 36000
processing 37000
processing 38000
processing 39000
processing 40000
processing 41000
processing 42000
processing 43000
train_x done
[0.038715824, 0.0054389383, 0.82709306, 0.005511425, 0.0029852896, 0.00064125628, 0.048602894, 0.014707477, 0.041944064, 0.014359762, 2]
2
processing 1000
processing 2000
test x done
[0.0066897073, 0.066193111, 0.0051110429, 0.018088473, 0.044554632, 0.00067090249,

In [4]:
import pickle
with open('../features/x_feat.pkl','wb') as fout:
    pickle.dump([x_data,y_data,test_x_data],fout)
print('feat done')

feat done
